# Embedding Matplotlib Animations in Jupyter as Interactive JavaScript Widgets

Written by Jean M Favre, Swiss National Supercomputing Center.
Tested on Piz Daint Sat May  1 15:31:28 CEST 2021

See http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/

On Piz Daint, a video can also be saved to disk using the FFmpeg module. We will use the new  module for v4.4 where we added the Theora encoder

put the following line in your $HOME/.jupyterhub.env

_module load FFmpeg_

using the _inline_ backend, the output of plotting commands is displayed directly below the code cell that produced it.

The resulting plots will then also be stored in the notebook document.
The image resolution defaults to 432x288 pixels, i.e. a figsize of 6x4, at 72 dpi

In [1]:
%matplotlib widget

In [2]:
import matplotlib
matplotlib.use("Agg")

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
#plt.rcParams['figure.figsize'] = [9.5, 6]

In [4]:
fig, ax = plt.subplots()

ax.set_xlim(( 0, 2))
ax.set_ylim((-2, 2))

line, = ax.plot([], [], lw=2)

In [5]:
def init():
    line.set_data([], [])
    return (line,)

In [6]:
def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return (line,)

In [7]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, 
                               blit=True)

By default, we don't get any sort of animation from the inline display

In [8]:
anim._repr_html_() is None

True

Change the default, to display the animation as HTML with JavaScript

In [9]:
rc('animation', html='jshtml')
anim

In [10]:
# An animation from begining to end can also be saved directly to disk as a video file
writervideo = animation.FFMpegWriter(fps=60, codec='theora') 
MovieFilename = "/dev/shm/matplotlibanimation.ogg"
anim.save(MovieFilename, writer=writervideo)

In [11]:
from ipywidgets import Video
video = Video.from_file(MovieFilename)
video

Video(value=b'OggS\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\n\x83\x9d\x19\x00\x00\x00\x009\xc9<\x10\x01*\x80the…

# The script above can be turned into a stand-alone Python script to execute on a terminal on Piz Daint

use the following:

_module load daint-gpu jupyter FFmpeg_

_jupyter nbconvert MatplotlibAnimation.ipynb --to python_

a file MatplotlibAnimation.py is generated.
Additionally, I comment out the first line of MatplotlibAnimation.py, i.e.
#get_ipython().run_line_magic('matplotlib', 'widget')

and I can run the whole thing as:

_python3 MatplotlibAnimation.py_